In [145]:
# Usage:
# This Python Notebook takes data from a real-time Kanda weather balloon launch CSV file and transforms it into the same
#   variables used as input into the ML model trained with IGRA data.
#
# The main goals are performed:
#   1. Removal of "ground" based observations
#   2. Calculation of lat/lon points into U and V wind components
#   3. Reformat CSV data into same DF format used as input into ML model.

In [146]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta

pd.set_option('display.max_rows', None)


In [147]:
import math

def haversine(coord1, coord2):

    R = 6372800  # Earth radius in meters

    lat1, lon1 = coord1
    lat2, lon2 = coord2

    phi1, phi2 = math.radians(lat1), math.radians(lat2)
    dphi       = math.radians(lat2 - lat1)
    dlambda    = math.radians(lon2 - lon1)

    a = math.sin(dphi/2)**2 + \
        math.cos(phi1)*math.cos(phi2)*math.sin(dlambda/2)**2

    return 2*R*math.atan2(math.sqrt(a), math.sqrt(1 - a))


#https://www.movable-type.co.uk/scripts/latlong.html

def direction(coord1, coord2):

    lat1, lon1 = coord1
    lat2, lon2 = coord2

    phi1, phi2 = math.radians(lat1), math.radians(lat2)

    lambda1, lambda2 = math.radians(lon1), math.radians(lon2);


    y = math.sin(lambda2 - lambda1) * math.cos(phi2);
    x = math.cos(phi1)*math.sin(phi2) - math.sin(phi1)*math.cos(phi2)*math.cos(lambda2 - lambda1);

    theta = math.atan2(y, x); # radians

    return theta


def wind_spd(coord1, coord2, time_delta_s):
    # Returns wind speed

    distance = haversine(coord1, coord2);
    return distance / time_delta_s; # m/s



def wind_direction(coord1, coord2):
    # Returns direction that wind is coming from
    
    wind_dir = direction(coord1, coord2);
    
    # Return direction in degrees where wind is coming FROM
    #  ... To get where wind is HEADED, instead use (math.degrees(dir) + 360) % 360

    return (math.degrees(wind_dir) +180) % 360;


In [148]:
df=pd.read_csv("ocw1k5zzkwml.csv")
df

,launch_id,unix_time_s,pressure_hpa,temperature_c,dewPoint_c,humidity_percent,latitude_deg,longitude_deg,wind_direction_deg,wind_speed_mps,elevation_gps_m,elevation2_m,flags
0,ocw1k5zzkwml,1632524785,991.900024,30.000000,NaN,26.500000,32.835636,-97.298965,NaN,NaN,179,0,205
1,ocw1k5zzkwml,1632524790,991.900024,30.110001,NaN,26.469999,32.835636,-97.298958,NaN,NaN,179,0,205
2,ocw1k5zzkwml,1632524795,991.900024,30.209999,NaN,26.190001,32.835636,-97.298950,NaN,NaN,179,0,205
3,ocw1k5zzkwml,1632524800,991.900024,30.299999,NaN,25.700001,32.835640,-97.298943,NaN,NaN,179,0,205
4,ocw1k5zzkwml,1632524805,991.900024,30.389999,NaN,25.290001,32.835644,-97.298943,NaN,NaN,186,0,205
5,ocw1k5zzkwml,1632524810,991.900024,30.490000,NaN,26.040001,32.835651,-97.298935,NaN,NaN,188,0,205
6,ocw1k5zzkwml,1632524815,991.900024,30.570000,NaN,26.610001,32.835651,-97.298935,NaN,NaN,189,0,205
7,ocw1k5zzkwml,1632524820,991.900024,30.660000,NaN,26.240000,32.835651,-97.298935,NaN,NaN,191,0,205
8,ocw1k5zzkwml,1632524825,991.900024,30.730000,NaN,25.770000,32.835659,-97.298950,NaN,NaN,193,0,205
9,ocw1k5zzkwml,1632524830,992.000000,30.799999,NaN,26.100000,32.835663,-97.298958,NaN,NaN,195,0,205


First we remove all datapoints located on the ground. 

These can be defined as any points with elevation2_m in the vacinity of 0-2 meters or close to 65535. The 65535 number is just an overflow value produced when negative values accidentally try to render as uint16 datatypes on the blockchain.

In [149]:
# Remove any values located on the ground
print(np.iinfo(np.uint16).max)
df = df[ (df['elevation2_m'] >= 3) & (df['elevation2_m'] < (np.iinfo(np.uint16).max - 3)) ]
df

65535


,launch_id,unix_time_s,pressure_hpa,temperature_c,dewPoint_c,humidity_percent,latitude_deg,longitude_deg,wind_direction_deg,wind_speed_mps,elevation_gps_m,elevation2_m,flags
123,ocw1k5zzkwml,1632525410,991.299988,30.070000,NaN,29.910000,32.835632,-97.298958,NaN,NaN,197,5,221
124,ocw1k5zzkwml,1632525415,988.400024,29.910000,NaN,24.990000,32.835743,-97.299034,NaN,NaN,215,31,221
125,ocw1k5zzkwml,1632525421,986.500000,29.750000,NaN,24.290001,32.836040,-97.298958,NaN,NaN,236,48,221
126,ocw1k5zzkwml,1632525426,984.799988,29.600000,NaN,23.920000,32.836353,-97.298996,NaN,NaN,255,64,221
127,ocw1k5zzkwml,1632525431,983.200012,29.469999,NaN,23.809999,32.836613,-97.299095,NaN,NaN,269,78,221
128,ocw1k5zzkwml,1632525436,981.099976,29.330000,NaN,23.809999,32.836891,-97.299110,NaN,NaN,288,97,221
129,ocw1k5zzkwml,1632525441,979.099976,29.200001,NaN,23.840000,32.837128,-97.299240,NaN,NaN,306,115,205
130,ocw1k5zzkwml,1632525446,976.599976,28.969999,NaN,24.000000,32.837410,-97.299248,NaN,NaN,326,138,205
131,ocw1k5zzkwml,1632525451,974.200012,28.709999,NaN,24.129999,32.837631,-97.299301,NaN,NaN,348,160,205
132,ocw1k5zzkwml,1632525456,972.599976,28.580000,NaN,24.219999,32.837891,-97.299324,NaN,NaN,364,175,205


In [150]:
# Here, we'll create a new pandas Dataframe resembling the relevant information from the CSV file.
#
# This new DF can be used as input into the ML model.

# Define function to convert unix timestamp to 12UTC datetime
def unixtime_to_launchtime(unix_time):
    
    timestamp_full = datetime.fromtimestamp( unix_time );
    
    # Get amount of time passed since 12UTC
    delta = timedelta( 
        hours=timestamp_full.hour - 12 , 
        minutes=timestamp_full.minute ,
        seconds=timestamp_full.second)
    
    # Return the difference.
    return timestamp_full - delta
    

# Create a new df frame that resembles model input
df_model = pd.DataFrame({'datetime': df["unix_time_s"].apply( lambda x: unixtime_to_launchtime(x) ),
                         'pressure': df["pressure_hpa"],
                         'temperature': df["temperature_c"],
                         'relativeHumidity': df["humidity_percent"]});
df_model

,datetime,pressure,temperature,relativeHumidity
123,2021-09-24 12:00:00,991.299988,30.070000,29.910000
124,2021-09-24 12:00:00,988.400024,29.910000,24.990000
125,2021-09-24 12:00:00,986.500000,29.750000,24.290001
126,2021-09-24 12:00:00,984.799988,29.600000,23.920000
127,2021-09-24 12:00:00,983.200012,29.469999,23.809999
128,2021-09-24 12:00:00,981.099976,29.330000,23.809999
129,2021-09-24 12:00:00,979.099976,29.200001,23.840000
130,2021-09-24 12:00:00,976.599976,28.969999,24.000000
131,2021-09-24 12:00:00,974.200012,28.709999,24.129999
132,2021-09-24 12:00:00,972.599976,28.580000,24.219999


We convert latitude and longitude measured from the GPS receiver into wind speed and direction, and then add those columns to our new dataframe.

In [151]:
# Define arrays to store U_wind and V_wind
U_wind = [];
V_wind = []

s_time = df['unix_time_s'].iloc[0];
end_time = df['unix_time_s'].iloc[-1];

# Calculate the U and V wind, given the latitude and longitude values reported by the GPS
#
# The reasoning here is that the balloon's location is directly proportional to the wind speed
#    in non-extreme wind shear conditions.
#
#   Theoretically, we would use the "flags" variable here to see if the GPS was accurate enough to use a smaller window of time
#      like 30 seconds instead.
#   For simplicity, we'll just take the 1-minute average wind.

# Loop over each observation
for ind in df.index:
    
    time = df['unix_time_s'][ind];
    
    # If too close to beginning or end of file
    if ( time - s_time < 30 or end_time - time < 30 ):
        U_wind.append(np.nan);
        V_wind.append(np.nan);
        continue;
    
    # Get starting grid point by looking back in time 30 seconds
    counter1 = 0;
    while ( (df['unix_time_s'][ind] - df['unix_time_s'][ind-counter1]) < 30 ):
        counter1 = counter1 + 1;
    lat_1 = df['latitude_deg'][ind-counter1];
    lon_1 = df['longitude_deg'][ind-counter1];
    
    
    # Get end grid point by looking forward in time 30 seconds
    counter2 = 0;
    while ( (df['unix_time_s'][ind+counter2] - df['unix_time_s'][ind] ) < 30 ):
        counter2 = counter2 + 1;
    lat_2 = df['latitude_deg'][ind+counter2];
    lon_2 = df['longitude_deg'][ind+counter2];
    
    # Get time difference
    dt = df['unix_time_s'][ind+counter2] - df['unix_time_s'][ind-counter1];
    
    # Calculate U and V wind (essentially wind speed in x and y direction)
    u_wind = wind_spd( [lat_1, lon_1] , [lat_1, lon_2], dt);
    if ( lon_2 < lon_1 ):
        u_wind = - u_wind;
    v_wind = wind_spd( [lat_1, lon_1] , [lat_2, lon_1], dt);
    if( lat_2 < lat_1 ):
        v_wind = - v_wind;
        
    U_wind.append(u_wind);
    V_wind.append(v_wind);
    
# Store the U and V wind into new columns
df_model["wind_U_component"] = U_wind;
df_model["wind_V_component"] = V_wind;

# NaNs at the beginning of the launch are typical. Not enough time has elapsed to get
#    an accurate reading on speed and direction
df = df_model

df

    


,datetime,pressure,temperature,relativeHumidity,wind_U_component,wind_V_component
123,2021-09-24 12:00:00,991.299988,30.070000,29.910000,NaN,NaN
124,2021-09-24 12:00:00,988.400024,29.910000,24.990000,NaN,NaN
125,2021-09-24 12:00:00,986.500000,29.750000,24.290001,NaN,NaN
126,2021-09-24 12:00:00,984.799988,29.600000,23.920000,NaN,NaN
127,2021-09-24 12:00:00,983.200012,29.469999,23.809999,NaN,NaN
128,2021-09-24 12:00:00,981.099976,29.330000,23.809999,NaN,NaN
129,2021-09-24 12:00:00,979.099976,29.200001,23.840000,-0.829897,5.543655
130,2021-09-24 12:00:00,976.599976,28.969999,24.000000,-0.771453,5.877527
131,2021-09-24 12:00:00,974.200012,28.709999,24.129999,-0.903142,5.841125
132,2021-09-24 12:00:00,972.599976,28.580000,24.219999,-0.903139,5.883555


Next we will derive atmospheric variables from the single launch, just like we would for IGRA2

After that, we can process it through our forecasting notebook(s)